In [1]:
import pandas as pd
import nltk
import numpy as np
import re
import os

nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/zacharychua/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

# 1. Data pre-processing

## 1.1. Data cleaning

In [2]:
# Load data
reviews_df = pd.read_csv('./output/company_reviews.csv')

In [3]:
# Check for na
reviews_df.isna().sum()

Unnamed: 0         0
company_name       0
date_posted     1850
rating             0
review_title       8
job_position       0
job_location       0
pro_review         0
con_review         0
dtype: int64

In [4]:
print(reviews_df['rating'].dtype)
reviews_df['rating'].unique()

float64


array([5., 3., 2., 4., 1.])

In [5]:
# Get count of unique company
pd.options.display.max_rows = 100
print('Total companies:',len(reviews_df['company_name'].unique()))
reviews_df['company_name'].value_counts()

Total companies: 90


company_name
NCS                                      190
Pactera                                  190
Seagate Technology                       189
Accenture                                170
Micron Technology                        170
Google                                    90
Standard Chartered Bank                   81
Works Applications                        59
Panasonic                                 56
Halliburton                               48
Amazon                                    42
MSD                                       35
Siemens                                   31
PSA International                         29
Hitachi                                   24
Anacle Systems                            23
WS Audiology                              22
Honeywell                                 20
Alcon                                     19
Shimizu                                   17
Marvell Technology                        17
Aurecon                                   

In [6]:
# Get count of unique job position
pd.options.display.max_rows = 1000
str(reviews_df['job_position'].unique()).replace('\n', '')

"['Senior Software Engineer' 'Software Engineer' 'Software Engineers' 'Software Engineer(Internship)' 'Cloud Associate' 'Data Engineer' 'Cloud Engineer' 'Lead Engineer' 'Solution Architect' 'Devops Engineer' 'Cloud Executive' 'Applications Engineer' 'Software Developer' 'Sr. DevOps Software Engineer' 'Senior Engineer' 'Application Engineer' 'Database Administrator' 'Seniour Software Engineer' 'Staff Software Engineer' 'DevOps Architect' 'Software Development Engineer In Test (SDET)' 'Lead Cloud Engineer' 'IT Infra Engineer' 'Technical Lead/Solution Architect' 'Software Enginner' 'Senior Cloud Engineer' 'Wintel Engineer' 'Engineer' 'Software EngineerSenior' 'Product Engineer' 'UAT Tester' 'Senior Applications Engineer' 'NLP Engineer Intern' 'Sr. Software Engineer' 'Software Engineer (Pega Developer)' 'Cloud Consultant' 'Solutions Architect' 'Senior Applications Developer' 'Computer Vision Intern' 'Software Testing Engineer' 'Test Analyst' 'Lead Sales Solution Architect' 'Senior Cloud En

# 1.2. Feature Engineering

### 1.2.1. Get sentiment score for each review

In [7]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

# Using vader to get sentiment score
def get_compound_score(review):
    return sid.polarity_scores(review)['compound']

# Getting compound score for "pro_review" and "con_review"
reviews_df['pro_score'] = reviews_df['pro_review'].apply(get_compound_score)
reviews_df['con_score'] = reviews_df['con_review'].apply(get_compound_score)

# Normalize rating
reviews_df['rating_score'] = reviews_df['rating'].apply(lambda r: (r-3) / 2)

# Calculating sentiment score
pro_score_weight = 0.25
con_score_weight = 0.25
rating_score_weight = 0.5
reviews_df['weighted_score'] = (pro_score_weight * reviews_df['pro_score']) + (con_score_weight * reviews_df['con_score']) + (rating_score_weight * reviews_df['rating_score'])

# Display the updated DataFrame
reviews_df.sort_values(by=['weighted_score'], ascending=False).head()

,Unnamed: 0,company_name,date_posted,rating,review_title,job_position,job_location,pro_review,con_review,pro_score,con_score,rating_score,weighted_score
1393,0,Marvell Technology,NaN,5.0,"Great place to work, many opportunities to grow",Senior Staff Engineer,Singapore,Interview process was smooth and all done onli...,"With many recent company acquisitions, there a...",0.9798,0.8266,1.0,0.951600
1734,0,Incube8,NaN,5.0,It’s been an amazing journey so far,Senior Automation Engineer,Singapore,Great learning experience - Lots to learn and ...,Very fast paced - Every release seems like a b...,0.9876,0.8100,1.0,0.949400
793,0,Seagate Technology,NaN,5.0,Good good,Senior Staff Engineer,Singapore,Good good good good good,Not bad not bad not bad,0.9260,0.8201,1.0,0.936525
547,0,Standard Chartered Bank,NaN,5.0,Great environment for work life balance,Cloud Engineer,Singapore,-Benefits are okay -Salary is great comparing ...,"Cant really think of any, but hopefully to get...",0.8225,0.8957,1.0,0.929550
848,0,Google,NaN,5.0,Very good,Senior Software Engineer,Singapore,innovative innovative innovative innovative in...,flexible flexible flexible flexible flexible,0.9260,0.7579,1.0,0.920975


### 1.2.2. Classify reviews into the following categories
* "Work life balance", "Progression", "Compensation and benefits", "Family oriented", "Personal development", "Culture" and "Diversity"

In [8]:
# Define keywords or phrases for each category to classify the reviews.
job_aspect_category_keywords = {
    "work_life_and_flexibility": [
        "work life balance", "no overtime", "flexible hours", "work-life", "work and life balance", 
        "remote work", "work from home", "telecommute", "flexible location", "work-life integration"
    ],
    "career_development_and_learning": [
        "career advancement", "promotion opportunities", "growth opportunities", "career growth", "progression", 
        "training", "courses", "skills development", "personal growth", "learning opportunities", 
        "career mobility", "lateral moves", "role changes", "internal opportunities", "innovation", 
        "creativity", "creative thinking", "innovative ideas"
    ],
    "compensation_benefits_and_security": [
        "salary", "pay", "compensation", "benefits", "bonus", "remuneration", "rewards", "insurance", 
        "retirement", "parental leave", "job security", "employment stability", "company stability", 
        "layoffs", "restructuring"
    ],
    "culture_and_environment": [
        "culture", "team spirit", "collaboration", "work environment", "company culture", 
        "team dynamics", "teamwork", "team collaboration", "team atmosphere", "environmental sustainability", 
        "eco-friendly", "sustainability efforts", "green initiatives", "health", "safety", 
        "well-being", "mental health", "workplace safety"
    ],
    "management_and_communication": [
        "leadership", "management", "executive", "supervisor", "managerial style", "communication", 
        "transparency", "open communication", "feedback"
    ],
    "diversity_and_inclusion": [
        "diversity", "inclusion", "equal opportunity", "inclusive environment", "cultural diversity", 
        "international", "multicultural"
    ],
    "employee_engagement_and_satisfaction": [
        "satisfaction", "engagement", "morale", "happiness", "valued", "team bonding", "job satisfaction"
    ],
    "operational_efficiency_and_resources": [
        "operational efficiency", "resources", "project management", "task management", "resource allocation", "organizational structure"
    ],
    "innovation_and_strategic_vision": [
        "company-wide innovation", "strategic direction", "future vision", "strategic planning", "long-term goals"
    ],
    "global_impact_and_social_responsibility": [
        "global impact", "social responsibility", "society contribution", "positive impact"
    ]
}

def is_category(review, keywords):
    return any(keyword in review.lower() for keyword in keywords)

def score_category(pro_review, pro_score, con_review, con_score, keywords):
    pro_score_ = pro_score if is_category(pro_review, keywords) else None
    con_score_ = con_score if is_category(con_review, keywords) else None
    if pro_score_ == None and con_score_ == None:
        return np.nan
    return (pro_score_ or 0) + (con_score_ or 0)

for category, keywords in job_aspect_category_keywords.items():
    # Apply job aspect classification to reviews
    reviews_df[f"{category}_job_aspect_score"] = reviews_df.apply(lambda row: score_category(
        pro_review=row.pro_review, con_review=row.con_review,
        pro_score=row.pro_score, con_score=row.con_score,
        keywords=keywords
    ), axis=1)

# Display the updated DataFrame
reviews_df.head()



,Unnamed: 0,company_name,date_posted,rating,review_title,job_position,job_location,pro_review,con_review,pro_score,...,work_life_and_flexibility_job_aspect_score,career_development_and_learning_job_aspect_score,compensation_benefits_and_security_job_aspect_score,culture_and_environment_job_aspect_score,management_and_communication_job_aspect_score,diversity_and_inclusion_job_aspect_score,employee_engagement_and_satisfaction_job_aspect_score,operational_efficiency_and_resources_job_aspect_score,innovation_and_strategic_vision_job_aspect_score,global_impact_and_social_responsibility_job_aspect_score
0,0,NCS,NaN,5.0,Good Company for learning different technologies,Senior Software Engineer,Singapore,You can learn different technologies if you ar...,"According to my view, no Cons about the company.",0.7650,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,NCS,NaN,3.0,"Good for stability, bad for growth",Software Engineer,Singapore,fresh grads get a bootcamp to ease them into c...,project you get assigned to may not be using t...,0.5859,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,NCS,NaN,2.0,bad proj,Software Engineers,"Ang Mo Kio New Town,",Mainly the benefits from the company.. Good le...,"Bad project, practices, management Colleagues ...",0.6705,...,NaN,NaN,0.6705,NaN,-0.5423,NaN,NaN,NaN,NaN,NaN
3,0,NCS,NaN,4.0,Good place to start,Software Engineer,"Novena,",Focus on software development as compare to so...,Fresh graduate will usually earn more than cur...,0.3612,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,NCS,NaN,4.0,good,Software Engineer,Singapore River,"work life balance, no ot","under payed, bonus very low",-0.2960,...,-0.296,NaN,0.2748,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 1.2.3. Classify job positions

In [9]:
# Update the keywords or phrases for job position categories.
job_position_category_keywords = {
    "software_engineer": ["software engineer", "developer", "programmer", "software architect", "frontend", "backend", "full stack"],
    "qa_automation_tester": ["qa", "quality assurance", "tester", "test engineer", "testing", "manual testing", "automation tester", "automated testing", "selenium", "test automation", "cypress"],
    "sre": ["site reliability", "sre", "reliability engineer"],
    "support_engineer": ["support engineer", "technical support"],
    "data_scientist_or_analyst": ["data scientist", "data analyst", "machine learning", "data engineer"],
    "product_manager": ["product manager", "product owner"],
    "ui_ux_designer": ["ui designer", "ux designer", "user interface", "user experience"],
    "network_engineer": ["network engineer", "network administrator", "networks"],
    "security_specialist": ["security", "cybersecurity", "information security"],
    "dev_ops_engineer": ["devops", "ci/cd", "automation engineer"],
    "cloud_engineer": ["cloud", "aws", "azure", "google cloud", "cloud engineer", "cloud architect"]
}

def classify_job_position(job_position, job_position_category_keywords):
    for category, keywords in job_position_category_keywords.items():
        if any(keyword in job_position.lower() for keyword in keywords):
            return category
    return "Non-tech related"  # Default to Non-tech related if no match found

# Apply job position classification to job positions
reviews_df['job_position_category'] = reviews_df['job_position'].apply(classify_job_position, args=(job_position_category_keywords,))

# Drop non-tech related reviews
reviews_df = reviews_df[reviews_df['job_position_category'] != "Non-tech related"]

# Display the updated DataFrame
reviews_df.head()

reviews_df.to_csv(f'./output/company_scores.csv')


### 1.2.4. Get freq for each category, grouped by company name and job position

In [10]:
job_aspect_category_columns = [f"{col}_job_aspect_score" for col in job_aspect_category_keywords.keys()]
keys = ['company_name', 'job_position_category']

job_aspect_category_columns += keys
freq_reviews_df = reviews_df.loc[:,job_aspect_category_columns].groupby(keys).count().add_suffix('_freq')

# Display the resultant DataFrame
freq_reviews_df.head()

work_life_and_flexibility_job_aspect_score_freq  \
company_name job_position_category                                                        
Accenture    cloud_engineer                                                           0   
             data_scientist_or_analyst                                                0   
             dev_ops_engineer                                                         1   
             qa_automation_tester                                                     2   
             software_engineer                                                       29   

                                        career_development_and_learning_job_aspect_score_freq  \
company_name job_position_category                                                              
Accenture    cloud_engineer                                                             0       
             data_scientist_or_analyst                                                  2       
             dev_ops_engineer                                                           0       
             qa_automation_tester                                                       2       
             software_engineer                                                         12       

                                        compensation_benefits_and_security_job_aspect_score_freq  \
company_name job_position_category                                                                 
Accenture    cloud_engineer                                                             0          
             data_scientist_or_analyst                                                  2          
             dev_ops_engineer                                                           0          
             qa_automation_tester                                                       3          
             software_engineer                                                         50          

                                        culture_and_environment_job_aspect_score_freq  \
company_name job_position_category                                                      
Accenture    cloud_engineer                                                         0   
             data_scientist_or_analyst                                              1   
             dev_ops_engineer                                                       0   
             qa_automation_tester                                                   3   
             software_engineer                                                     14   

                                        management_and_communication_job_aspect_score_freq  \
company_name job_position_category                                                           
Accenture    cloud_engineer                                                             1    
             data_scientist_or_analyst                                                  1    
             dev_ops_engineer                                                           0    
             qa_automation_tester                                                       2    
             software_engineer                                                         16    

                                        diversity_and_inclusion_job_aspect_score_freq  \
company_name job_position_category                                                      
Accenture    cloud_engineer                                                         0   
             data_scientist_or_analyst                                              0   
             dev_ops_engineer                                                       0   
             qa_automation_tester                                                   0   
             software_engineer                                                      2   

                                        employee_engagement_and_satisfaction_job_aspect_score_freq  \
company_name

### 1.2.5. Get mean score for each category, grouped by company name and job position


In [11]:
score_columns = [col for col in reviews_df.columns if re.match(r"\w+_score$", col)]
keys = ['company_name', 'job_position_category']

score_columns += ['company_name', 'job_position_category']
mean_reviews_df = reviews_df.loc[:,score_columns].groupby(keys).mean().add_suffix('_mean')

# Display the updated DataFrame
mean_reviews_df.head()


pro_score_mean  con_score_mean  \
company_name job_position_category                                       
Accenture    cloud_engineer                   0.213775        0.161325   
             data_scientist_or_analyst        0.381120       -0.023400   
             dev_ops_engineer                 0.440400        0.000000   
             qa_automation_tester             0.609114       -0.123300   
             software_engineer                0.515833       -0.091590   

                                        rating_score_mean  \
company_name job_position_category                          
Accenture    cloud_engineer                      0.625000   
             data_scientist_or_analyst           0.700000   
             dev_ops_engineer                    0.750000   
             qa_automation_tester                0.642857   
             software_engineer                   0.271552   

                                        weighted_score_mean  \
company_name job_position_category                            
Accenture    cloud_engineer                        0.406275   
             data_scientist_or_analyst             0.439430   
             dev_ops_engineer                      0.485100   
             qa_automation_tester                  0.442882   
             software_engineer                     0.241837   

                                        work_life_and_flexibility_job_aspect_score_mean  \
company_name job_position_category                                                        
Accenture    cloud_engineer                                                         NaN   
             data_scientist_or_analyst                                              NaN   
             dev_ops_engineer                                                  0.440400   
             qa_automation_tester                                              0.657850   
             software_engineer                                                -0.144828   

                                        career_development_and_learning_job_aspect_score_mean  \
company_name job_position_category                                                              
Accenture    cloud_engineer                                                           NaN       
             data_scientist_or_analyst                                           0.315000       
             dev_ops_engineer                                                         NaN       
             qa_automation_tester                                               -0.146200       
             software_engineer                                                   0.466025       

                                        compensation_benefits_and_security_job_aspect_score_mean  \
company_name job_position_category                                                                 
Accenture    cloud_engineer                                                           NaN          
             data_scientist_or_analyst                                           0.234100          
             dev_ops_engineer                                                         NaN          
             qa_automation_tester                                                0.656933          
             software_engineer                                                   0.252772          

                                        culture_and_environment_job_aspect_score_mean  \
company_name job_position_category                                                      
Accenture    cloud_engineer                                                       NaN   
             data_scientist_or_analyst                                       0.807400   
             dev_ops_engineer                                                     NaN   
             qa_automation_tester                                            0.446133   
             software_engineer                                               0.351193   

        

### 1.2.6. Combine dataframe

In [12]:
final_df = pd.concat([freq_reviews_df, mean_reviews_df], axis=1)

# Display the final output
final_df.head()

work_life_and_flexibility_job_aspect_score_freq  \
company_name job_position_category                                                        
Accenture    cloud_engineer                                                           0   
             data_scientist_or_analyst                                                0   
             dev_ops_engineer                                                         1   
             qa_automation_tester                                                     2   
             software_engineer                                                       29   

                                        career_development_and_learning_job_aspect_score_freq  \
company_name job_position_category                                                              
Accenture    cloud_engineer                                                             0       
             data_scientist_or_analyst                                                  2       
             dev_ops_engineer                                                           0       
             qa_automation_tester                                                       2       
             software_engineer                                                         12       

                                        compensation_benefits_and_security_job_aspect_score_freq  \
company_name job_position_category                                                                 
Accenture    cloud_engineer                                                             0          
             data_scientist_or_analyst                                                  2          
             dev_ops_engineer                                                           0          
             qa_automation_tester                                                       3          
             software_engineer                                                         50          

                                        culture_and_environment_job_aspect_score_freq  \
company_name job_position_category                                                      
Accenture    cloud_engineer                                                         0   
             data_scientist_or_analyst                                              1   
             dev_ops_engineer                                                       0   
             qa_automation_tester                                                   3   
             software_engineer                                                     14   

                                        management_and_communication_job_aspect_score_freq  \
company_name job_position_category                                                           
Accenture    cloud_engineer                                                             1    
             data_scientist_or_analyst                                                  1    
             dev_ops_engineer                                                           0    
             qa_automation_tester                                                       2    
             software_engineer                                                         16    

                                        diversity_and_inclusion_job_aspect_score_freq  \
company_name job_position_category                                                      
Accenture    cloud_engineer                                                         0   
             data_scientist_or_analyst                                              0   
             dev_ops_engineer                                                       0   
             qa_automation_tester                                                   0   
             software_engineer                                                      2   

                                        employee_engagement_and_satisfaction_job_aspect_score_freq  \
company_name

In [13]:
final_df.to_csv(f"./output/company_scores_aggregate.csv")

In [14]:
final_df.index

MultiIndex([(           'Accenture',            'cloud_engineer'),
            (           'Accenture', 'data_scientist_or_analyst'),
            (           'Accenture',          'dev_ops_engineer'),
            (           'Accenture',      'qa_automation_tester'),
            (           'Accenture',         'software_engineer'),
            (               'Alcon',      'qa_automation_tester'),
            (       'Alibaba Group',         'software_engineer'),
            (              'Amazon',            'cloud_engineer'),
            (              'Amazon', 'data_scientist_or_analyst'),
            (              'Amazon',          'dev_ops_engineer'),
            ...
            ('Trading Technologies',                       'sre'),
            (        'Transit Link',          'support_engineer'),
            (              'VMware',         'software_engineer'),
            (            'Veracode',         'software_engineer'),
            (        'WS Audiology',         '